# Tookit

In [1]:
# import modules
import numpy as np
import csv
import networkx as nx
import time
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import multiprocessing as mp
from scipy.integrate import odeint

In [2]:
_Data_PATH_ = './data/' # where to save the data
_Figure_PATH_ = './figures/' # where to save the figures

In [3]:
# define colors
colors = ['C0', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

## Network Structure

In [4]:
# generate a network structure
def network(param_nx, structure = 'complete'):
    '''
    param_nx: dictionary of parameters
    structure: type of network
    '''
    if structure == 'complete': # complete graph
        G = nx.complete_graph(param_nx['N'])
    elif structure == 'lattice': # two-dimensional grid graph (square or rectangle)
        G = nx.grid_2d_graph(param_nx['L'], param_nx['H'], periodic = True) 
    elif structure == 'ER': # Erdős-Rényi graph
        G = nx.erdos_renyi_graph(param_nx['N'], param_nx['p'])
    elif structure == 'WS': # Watts–Strogatz small-world graph
        G = nx.watts_strogatz_graph(param_nx['N'], param_nx['k'], param_nx['b'])
    elif structure == 'BA': # Barabási–Albert preferential attachment
        G = nx.barabasi_albert_graph(param_nx['N'], param_nx['m'])

    print(nx.info(G)) # basic information
    return nx.to_numpy_array(G) # adjacency matrix

In [5]:
# param_nx = {'N': 1000, 'L': 30, 'H': 30, 'p': 0.02, 'k': 20, 'b': 0.2, 'm': 10}
# names = ['complete', 'lattice', 'ER', 'WS', 'BA']
# A_complete = network(param_nx, structure = names[0])
# A_lattice = network(param_nx, structure = names[1])
# A_ER = network(param_nx, structure = names[2])
# A_WS = network(param_nx, structure = names[3])
# A_BA = network(param_nx, structure = names[4])

## Epidemiological model

* ### well-mixed population

    * #### ODE equation
    * #### Gillespie algorithm
    * #### $\tau$-leaping algorithm

* ### structured population

    * #### Gillespie algorithm

In [6]:
# well-mixed population
## ODE equation
### SIR model: numerical solution
def SIR_equation(y, t, param):
    beta, gamma = param['beta'], param['gamma']
    S, I, R = y
    return [-beta*S*I, beta*S*I - gamma*I, gamma*I]
def SIR(y_0, t, param):
    solution = odeint(SIR_equation, y_0, t, args = (param, ))
    return solution

In [7]:
## Gillespie algorithm
### SIR model: agent-based simulation
def SIR_Gillespie(N, y_0, param, tag = 'SIR'):
    '''
    N: population size
    y_0: initial condition of the population
    param: dictionary of the parameters
    tag: output object
    '''
    # initialization
    beta, gamma = param['beta']/N, param['gamma']
    I_0 = int(N*y_0[1])
    t = [0]; S = [N - I_0]; I = [I_0]; R = [0]
    state = np.array([0]*S[0] + [1]*I[0]).reshape((N, )) # S: 0, I: 1, R: 2
    np.random.shuffle(state)
    prop = beta*I[0]*(state == 0) + gamma*(state == 1)
    
    # update
    while I[-1] > 0:
        r_1 = np.random.rand()
        r_2 = np.random.rand()
        cumsum = prop.cumsum()
        scale = cumsum[-1] # total rate, or beta*S[-1]*I[-1] + gamma*I[-1]
        tau = -np.log(1 - r_1)/scale # exponential distribution: inverse function
        t.append(t[-1] + tau)

        i = np.searchsorted(cumsum, r_2*scale)
        if state[i] == 0: 
            S.append(S[-1] - 1); I.append(I[-1] + 1); R.append(R[-1])
            state[i] = 1 # S to I
            prop[i] = gamma
        else:
            S.append(S[-1]); I.append(I[-1] - 1); R.append(R[-1] + 1)
            state[i] = 2 # I to R
            prop[i] = 0
        prop[state == 0] = beta*I[-1]
        
    if tag == 'SIR':
        S = np.array(S); I = np.array(I); R = np.array(R)
        solution = np.column_stack((S, I, R))
        return t, solution 
    else:
        return state

In [8]:
## tau-leaping algorithm
### SIR model
def SIR_tau_leaping(N, y_0, param):
    '''
    N: population size
    y_0: initial condition of the population
    param: dictionary of the parameters
    '''
    # initialization
    beta, gamma = param['beta']/N, param['gamma']
    I_0 = int(N*y_0[1])
    t = [0]; S = [N - I_0]; I = [I_0]; R = [0]
    tau = 0.01
    # update
    while I[-1] > 0:
        lam_1 = beta*S[-1]*I[-1]*tau
        lam_2 = gamma*I[-1]*tau
        K_1 = np.random.poisson(lam_1)
        K_2 = np.random.poisson(lam_2)
        S.append(S[-1] - K_1); I.append(I[-1] + K_1 - K_2); R.append(R[-1] + K_2)
        t.append(t[-1] + tau)
    S = np.array(S); I = np.array(I); R = np.array(R)
    solution = np.column_stack((S, I, R))
    return t, solution

In [9]:
# # SIR
# N = 1000; y_0 = [1 - 1e-2, 1e-2, 0]; t = np.linspace(0, 500, 10001); param = {'beta': 0.35, 'gamma': 0.1}
# solution = SIR(y_0, t, param); solution = np.around(solution, decimals = 6)
# print('ODE: ' + ", ".join([str(v) for v in solution[-1]]))
# repetition = 100
# for fun in [SIR_Gillespie, SIR_tau_leaping]:
#     start = time.time()
#     mean_SIR = np.zeros(3)
#     for i in range(repetition):
#         t, solution = fun(N, y_0, param); mean_SIR += solution[-1]
#     mean_SIR = mean_SIR/repetition/N; mean_SIR = np.around(mean_SIR, decimals = 6)
#     end = time.time()
#     print(fun.__name__[4:] + ': ' + ", ".join([str(v) for v in mean_SIR]))
#     print('time: ' + str(np.around(end - start, decimals = 6)))

## Figure

In [10]:
def figure_disease(N, y_0, t, param, fsize = (8, 6), fs = 20, model = 'SIR', method = 'ODE'):
    '''
    N: population size
    y_0: initial condition of the population
    param: dictionary of the parameters
    model: SIR
    method: ODE or Gillespie or tau-leaping
    '''
    funs = {'SIR model: ODE': SIR, 
            'SIR model: Gillespie': SIR_Gillespie, 
            'SIR model: tau-leaping': SIR_tau_leaping}
    name = model + ' model: ' + method
    fun = funs[name]
    if method == 'ODE':
        solution = fun(y_0, t, param)
    else:
        t, solution = fun(N, y_0, param)
        solution = solution/N
    
    fig = plt.figure(figsize = fsize)
    fig.patch.set_facecolor('white')
    ax = plt.subplot(111)
    ax.set_xlabel('time', fontsize = fs - 2)
    ax.set_ylabel('fraction of individuals', fontsize = fs - 2)
    ax.tick_params(axis = 'both', which = 'major', labelsize = fs - 4)
    ax.tick_params(axis = 'both', which = 'minor', labelsize = fs - 4)
    ax.set_xlim(0, 100)
    ax.set_ylim(-0.1, round(np.max(solution), 1) + 0.1)
    cs = {'S': colors[0], 'V': colors[4], 'I': colors[3], 'R': colors[2]}
    if model == 'SIR':
        labels = ['S', 'I', 'R']
        title = 'SIR model: '
    else:
        labels = ['S', 'V', 'I', 'R']
        title = 'SIRV model: '
    for i, lab in enumerate(labels):
        plt.plot(t, solution[:, i], color = cs[lab], linewidth = 3, label = lab)
    ax.legend(loc = 'upper right', ncol = solution.shape[1], fancybox = True, fontsize = fs - 4)
    plt.title(title + method, fontsize = fs, y = 1.02)
    plt.show()

In [11]:
# # example
# N = 5000; y_0 = [1 - 1e-2, 1e-2, 0]; t = np.linspace(0, 100, 1001); param = {'beta': 0.35, 'gamma': 0.1}
# figure_disease(N, y_0, t, param, fsize = (6, 4), fs = 15, model = 'SIR', method = 'ODE')
# figure_disease(N, y_0, t, param, fsize = (6, 4), fs = 15, model = 'SIR', method = 'Gillespie')
# figure_disease(N, y_0, t, param, fsize = (6, 4), fs = 15, model = 'SIR', method = 'tau-leaping')

In [12]:
# structured population
## SIR model: agent-based simulation
def SIR_network_Gillespie(A, y_0, param, output = 'solution'):
    '''
    A: adjacency matrix
    y_0: initial condition of the population
    param: dictionary of the parameters
    output: output object
    '''
    # initialization
    N = A.shape[0]; degree = np.sum(A)/A.shape[0]
    beta, gamma = param['beta']/degree, param['gamma']
    I_0 = int(N*y_0[1])
    t = [0]; S = [N - I_0]; I = [I_0]; R = [0]
    state = np.array([0]*S[0] + [1]*I[0]).reshape((N, ))
    np.random.shuffle(state)
    # compared with well-mixed population: beta*I[0]*(state == 0) + gamma*(state == 1)
    prop = beta*A.dot(state == 1)*(state == 0) + gamma*(state == 1) 
    # update
    while I[-1] > 0:
        r_1 = np.random.rand()
        r_2 = np.random.rand()
        cumsum = prop.cumsum()
        scale = cumsum[-1]
        tau = -np.log(1 - r_1)/scale 
        t.append(t[-1] + tau)
        i = np.searchsorted(cumsum, r_2*scale)
        if state[i] == 0: 
            S.append(S[-1] - 1); I.append(I[-1] + 1); R.append(R[-1])
            state[i] = 1
        else:
            S.append(S[-1]); I.append(I[-1] - 1); R.append(R[-1] + 1)
            state[i] = 2 
        # update propensities (neighborhood affected)  
        prop = beta*A.dot(state == 1)*(state == 0) + gamma*(state == 1) 
        
    if output == 'solution':
        S = np.array(S); I = np.array(I); R = np.array(R)
        solution = np.column_stack((S, I, R))
        return t, solution
    else:
        return state

In [13]:
# # SIR
# N = 1000; A = network({'N': N}, structure = 'complete')
# y_0 = [1 - 1e-2, 1e-2, 0]; param = {'beta': 0.35, 'gamma': 0.1}
# repetition = 100
# start = time.time()
# mean_SIR = np.zeros(3)
# for i in range(repetition):
#     t, solution = SIR_network_Gillespie(A, y_0, param, output = 'solution'); mean_SIR += solution[-1]
# mean_SIR = mean_SIR/repetition/N; mean_SIR = np.around(mean_SIR, decimals = 6)
# end = time.time()
# print('network_Gillespie' + ': ' + ", ".join([str(v) for v in mean_SIR]))
# print('time: ' + str(np.around(end - start, decimals = 6)))

## Object Oriented Programming


### well-mixed population
- [x] ODE equation 
```python
model.equation()
```
- [x] Gillespie algorithm
```python
model.simulation(tag = 'all_wm')
model.simulation(tag = 'fancy_wm')
```

### structured population

- [x] Gillespie algorithm
```python
model.simulation(tag = 'all')
model.simulation(tag = 'take')
model.simulation(tag = 'fancy')
```

In [14]:
# SIR
class SIR_model():
    """
    a class to simulate the SIR Model
    A: network adjacency matrix or network graph object
    y_0: initial condition of the population
    param: dictionary of the parameters
    tag: method of array manipulation
    """
    def __init__(self, A, y_0, param, tag):
        # network
        self.A = A
        # parameters
        self.N = A.shape[0]
        self.degree = np.sum(A)/A.shape[0]
        self.beta = param['beta']/self.degree
        self.gamma = param['gamma']
        self.tag = tag
        # time 
        self.t = [0]
        # fractions of individuals
        self.S = [self.N - int(self.N*y_0[1])]
        self.I = [int(self.N*y_0[1])]
        self.R = [0]
        # state
        self.state = np.array([0]*self.S[0] + [1]*self.I[0]).reshape((self.N, ))
        np.random.shuffle(self.state)
        # propensity 
        if 'wm' in self.tag: # well-mixed
            self.ALL_WM()
        else:
            self.ALL()
        # which propensity scheme to use
        if self.tag == 'all_wm':
            self.UPDATE = self.ALL_WM
        elif self.tag == 'fancy_wm':
            self.UPDATE = self.FANCY_WM
        elif self.tag == 'all':
            self.UPDATE = self.ALL
        elif self.tag == 'take':
            self.UPDATE = self.TAKE
        elif self.tag == 'fancy':
            self.UPDATE = self.FANCY
        return None
    
    def ALL_WM(self, nb = None):
        self.prop = self.beta*self.I[-1]*(self.state == 0) + self.gamma*(self.state == 1)
        return None
    def FANCY_WM(self, nb):
        self.prop[nb] = self.beta*self.I[-1]
        return None
    def ALL(self, nb = None):   
        self.prop = self.beta*self.A.dot(self.state == 1)*(self.state == 0) + self.gamma*(self.state == 1)
        return None
    def TAKE(self, nb):
        #self.prop[nb] = self.beta*np.take(self.A, nb, axis = 0).dot(self.state == 1) # consider only susceptible
        self.prop[nb] = (self.beta*np.take(self.A, nb, axis = 0).dot(self.state == 1)
                         *(np.take(self.state, nb) == 0) + self.gamma*(np.take(self.state, nb) == 1))
        return None
    def FANCY(self, nb):
        #self.prop[nb] = self.beta*self.A[nb].dot(self.state == 1) # consider only susceptible
        self.prop[nb] = (self.beta*self.A[nb].dot(self.state == 1)*(self.state[nb] == 0) + 
                         self.gamma*(self.state[nb] == 1))
        return None
    
    def iteration(self):
        # termination
        if self.I[-1] == 0:
            self.S = np.array(self.S, dtype = float)
            self.I = np.array(self.I, dtype = float)
            self.R = np.array(self.R, dtype = float)
            self.solution = np.column_stack((self.S, self.I, self.R))
            return False

        # 1. generate random numbers r_1, r_2 uniformly distributed in (0, 1)
        r_1 = np.random.rand()
        r_2 = np.random.rand()
        # 2. calculate scale
        cumsum = self.prop.cumsum()
        self.scale = cumsum[-1]
        # 3. compute the time until the next reaction takes place
        tau = -np.log(float(1 - r_1))/self.scale
        self.t.append(self.t[-1] + tau)
        # 4. figure out which reaction takes place
        i = np.searchsorted(cumsum, r_2*self.scale)
        # 5. update node states
        if self.state[i] == 0:
            self.S.append(self.S[-1] - 1); self.I.append(self.I[-1] + 1); self.R.append(self.R[-1])
            self.state[i] = 1
            self.prop[i] = self.gamma
        else:
            self.S.append(self.S[-1]); self.I.append(self.I[-1] - 1); self.R.append(self.R[-1] + 1)
            self.state[i] = 2  
            self.prop[i] = 0
        # 6. update propensities
        if 'all' in self.tag:
            self.UPDATE()
        else:
            if 'wm' in self.tag:
                nb = np.nonzero(self.state == 0)[0]
            else:
                #nb = np.nonzero(self.A[i]*(self.state == 0))[0] # !!! time consuming !!!
                nb = np.nonzero(self.A[i])[0]
            self.UPDATE(nb)  
        return True

    def simulation(self):
        running = True
        while running:
            running = self.iteration()
        return None
    
    def equation(self):
        beta = self.beta*self.degree
        gamma = self.gamma
        def derivative(y, t):
            S, I, R = y
            return [-beta*S*I, beta*S*I - gamma*I, gamma*I]
        y_0 = [float(self.S[0])/self.N, float(self.I[0])/self.N, float(self.R[0])/self.N]
        t = np.linspace(0, max(self.t[-1], 5000), 1001)
        self.solution = odeint(derivative, y_0, t)
        return None       

### Example 1: Complete Graph

In [15]:
N = 1000; A = network({'N': N}, structure = 'complete')
y_0 = [1 - 0.01, 0.01, 0]; param = {'beta': 0.35, 'gamma': 0.1}

C:\Users\93217\AppData\Local\Temp\ipykernel_24956\4094712956.py:18: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G)) # basic information


Graph with 1000 nodes and 499500 edges


In [ ]:
model = SIR_model(A, y_0, param, tag = 'all_wm'); model.simulation(); model.equation()
solution = np.around(model.solution, decimals = 6)[-1]
print('ODE: ' + ", ".join([str(v) for v in solution]))
repetition = 100
for tag in ['all_wm', 'fancy_wm', 'all', 'take', 'fancy']:
    start = time.time()
    mean_SIR = np.zeros(3)
    for i in range(repetition):
        model = SIR_model(A, y_0, param, tag = tag); model.simulation()
        solution = model.solution[-1]; mean_SIR += solution
    mean_SIR = mean_SIR/repetition/N; mean_SIR = np.around(mean_SIR, decimals = 6)
    end = time.time()
    print(tag + ': ' + ", ".join([str(v) for v in mean_SIR]))
    print('time: ' + str(np.around(end - start, decimals = 6)))

ODE: 0.03363, -0.0, 0.96637
all_wm: 0.0352, 0.0, 0.9648
time: 3.780152
fancy_wm: 0.03356, 0.0, 0.96644
time: 3.090533
all: 0.0339, 0.0, 0.9661
time: 37.399338


### Example 2: Marvel Universe

In [ ]:
with open('hero-network1.csv', newline = '') as file:
    reader = csv.reader(file)
    next(reader, None) # skip the headers
    data = list(reader)
G = nx.from_edgelist(data)
A = nx.to_numpy_array(G)
N = A.shape[0]
print(nx.info(G))

In [ ]:
y_0 = [1 - 0.01, 0.01, 0]; param = {'beta': 0.35, 'gamma': 0.1}
repetition = 10
for tag in ['all', 'take', 'fancy']:
    start = time.time()
    mean_SIR = np.zeros(3)
    for i in range(repetition):
        model = SIR_model(A, y_0, param, tag = tag); model.simulation()
        solution = model.solution[-1]; mean_SIR += solution
    mean_SIR = mean_SIR/repetition/N; mean_SIR = np.around(mean_SIR, decimals = 6)
    end = time.time()
    print(tag + ': ' + ", ".join([str(v) for v in mean_SIR]))
    print('time: ' + str(np.around(end - start, decimals = 6)))